In [4]:
import pandas as pd
import os 
os.chdir('/mnt/data/hong/courses/varianceAnalysis/')

In [38]:
obs = pd.read_csv('data/ADA_variant_clinic.tsv', sep='\t')

In [6]:
fathmm = pd.read_csv('data/fathmm_from_ADA_variants_rs.tab', sep='\t')

In [9]:
amis = pd.read_csv('data/ADA_variant_rsid_vep_aMissense.txt', sep='\t', skiprows=[*range(44)])

In [39]:
## merge the dataframes
obs_fathmm = pd.merge(obs, fathmm, left_on= 'id', right_on='dbSNP ID', how='left')
obs_fathmm_amis = pd.merge(obs_fathmm, amis, left_on= 'id', right_on='#Uploaded_variation', how='left')
obs_fathmm_amis.fillna('NA', inplace=True)

In [41]:
## FIsher exact test

obs_clinicalSignificances = obs_fathmm_amis['clinicalSignificances']
fathmm_clinicalSignificances = obs_fathmm_amis['Prediction']
## split the column extra by semicomma
## use regex to extract the string after IMPACT= and before ; in Extra column
obs_fathmm_amis['IMPACT'] = obs_fathmm_amis['Extra'].str.extract(r'IMPACT=(.*?);')
amis_clinicalSignificances = obs_fathmm_amis['IMPACT']
## test the association between clinical significance and fathmm prediction



In [47]:
from scipy.stats import contingency
fathmm_crosstab = pd.crosstab(obs_fathmm_amis['clinicalSignificances'], obs_fathmm_amis['Prediction'])
fathmm_crosstab_rec = fathmm_crosstab.drop('NA', axis=0)
contingency.chi2_contingency(fathmm_crosstab_rec)

(2393.111833770712,
 0.0,
 3,
 array([[ 343.97731959,  595.02268041],
        [  56.78006873,   98.21993127],
        [ 666.34158076, 1152.65841924],
        [1064.90103093, 1842.09896907]]))

In [50]:
amis_crosstab = pd.crosstab(obs_fathmm_amis['clinicalSignificances'], obs_fathmm_amis['IMPACT'])
amis_crosstab_rec = amis_crosstab.drop('NA', axis=0)
contingency.chi2_contingency(amis_crosstab_rec)

(412.55607384818313,
 2.8636735807461966e-83,
 9,
 array([[5.64690722e+00, 4.19484536e+01, 3.90120619e+02, 5.01284021e+02],
        [9.32130584e-01, 6.92439863e+00, 6.43969072e+01, 8.27465636e+01],
        [1.09390034e+01, 8.12611684e+01, 7.55728866e+02, 9.71070962e+02],
        [1.74819588e+01, 1.29865979e+02, 1.20775361e+03, 1.55189845e+03]]))

In [66]:
## correlation with the MAF and HMM weights D.
## remove the rows where MAF is NA
obs_fathmm_amis_rec = obs_fathmm_amis[obs_fathmm_amis['MAF'] != 'NA']

obs_fathmm_amis_rec['HMM Prob. M._rec'] = obs_fathmm_amis_rec['HMM Prob. M.'].replace('NA', '0')
## 0 meaning no change, as a guess when you don't have the data
## pearson correlation
obs_fathmm_amis_rec['MAF'] = obs_fathmm_amis_rec['MAF'].astype(float)
obs_fathmm_amis_rec['HMM Prob. M._rec'] = obs_fathmm_amis_rec['HMM Prob. M._rec'].astype(float)
obs_fathmm_amis_rec['MAF'].corr(obs_fathmm_amis_rec['HMM Prob. M._rec'])
## linear regression
from scipy.stats import linregress
linregress(obs_fathmm_amis_rec['MAF'], obs_fathmm_amis_rec['HMM Prob. M._rec'])





/tmp/ipykernel_3864203/3580885277.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  obs_fathmm_amis_rec['HMM Prob. M._rec'] = obs_fathmm_amis_rec['HMM Prob. M.'].replace('NA', '0')
/tmp/ipykernel_3864203/3580885277.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  obs_fathmm_amis_rec['MAF'] = obs_fathmm_amis_rec['MAF'].astype(float)
/tmp/ipykernel_3864203/3580885277.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

LinregressResult(slope=1.0285717243939256, intercept=0.011748160030491048, rvalue=0.6230381134654781, pvalue=2.7072513044468046e-202, stderr=0.02982170094801998, intercept_stderr=0.0006007061203695877)

In [63]:
obs_fathmm_amis_rec['HMM Prob. M._rec']

288      NaN
289      NaN
290      NaN
291      NaN
292      NaN
        ... 
17830    0.0
17831    0.0
17832    0.0
17833    0.0
17834    0.0
Name: HMM Prob. M._rec, Length: 1877, dtype: float64